In [1]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="phpipamadmin",
  database="assetdb"
)

mycursor = mydb.cursor()
mycursor.execute("SHOW TABLES")
for x in mycursor:
    print(x)
mycursor.close()

True

In [2]:
mycursor = mydb.cursor()
mycursor.execute("CREATE TABLE p_netflow (srcaddr INT UNSIGNED, srcport MEDIUMINT UNSIGNED, first BIGINT UNSIGNED, last BIGINT UNSIGNED, protocol TINYINT UNSIGNED, flows SMALLINT UNSIGNED, packets MEDIUMINT UNSIGNED, bytes INT UNSIGNED, PRIMARY KEY(`srcaddr`)) \
ENGINE=InnoDB DEFAULT CHARSET=utf8 PARTITION BY HASH(srcaddr) partitions 64;")
mycursor.close()

True

In [3]:
import socket  
import struct  
  

ip = '127.0.0.1'  
int_ip = struct.unpack('!I', socket.inet_aton(ip))[0]  
print(int_ip)  
str_ip = socket.inet_ntoa(struct.pack('!I', int_ip))  
print(str_ip) 


2130706433
127.0.0.1


In [4]:
def protocol_to_number(protocol):
    if protocol == 'TCP':
        num = 1
    elif protocol=='UDP':
        num = 2
    elif protocol == 'ssh':
        num = 3
    return num

In [5]:
# Python read all files from a directory in order
import os

f = open("./data/netflow_name.txt", 'w')
path = "./data/netflow/"
files = os.listdir(path)
files.sort()

s = []

for file_name in files:
    if not os.path.isdir(path + file_name):
        f_name = str(file_name)
        print(f_name)
        s.append(f_name)
        f.write(f_name + '\n')
for i in s:
    print(i)
print(s)

20200528140758.json
20200528141325.json
20200528141851.json
20200528142416.json
20200528142940.json
20200528143504.json
20200528144029.json
20200528144554.json
20200528145119.json
20200528145643.json
20200528150208.json
20200528150732.json
20200528151257.json
20200528151821.json
20200528152346.json
20200528152910.json
20200528153435.json
20200528153959.json
20200528154524.json
20200528155048.json
20200528155613.json
20200528160137.json
20200528160702.json
20200528161228.json
20200528161752.json
20200528162317.json
20200528162841.json
20200528163405.json
20200528163930.json
20200528164454.json
20200528165019.json
20200528165543.json
20200528170107.json
20200528170632.json
20200528171156.json
20200528171721.json
20200528172246.json
20200528172810.json
20200528173335.json
20200528173859.json
20200528174424.json
20200528174948.json
20200528175513.json
20200528180037.json
20200528180602.json
20200528181126.json
20200528181650.json
20200528182215.json
20200528182740.json
20200528183304.json


In [ ]:
import time
import json

# update 1 X 1000000 records at a time

for json_name in s:
    mydb.reconnect()
    mycursor = mydb.cursor()
    sql_update = "INSERT INTO p_netflow (srcaddr, srcport, first, last, protocol, flows, packets, bytes) VALUES (%s,%s,%s,%s,%s,%s,%s,%s) ON DUPLICATE KEY UPDATE `last` = VALUES(`last`), `flows`= VALUES(`flows`) + `flows`, `packets`= VALUES(`packets`) + `packets`, `bytes`= VALUES(`bytes`) + `bytes`"

    start_time = time.time()
    flows = []
    with open('data/netflow/' + json_name) as record:
        new_record = json.load(record)
        new_record.sort(key=lambda k: (k.get('srcaddr', 0)))
        for flow in new_record:
            srcaddr = struct.unpack('!I', socket.inet_aton(flow["srcaddr"]))[0]
            protocol = protocol_to_number(flow["protocol"])
            flow_one = (srcaddr, flow["srcport"], flow["First"], flow["Last"], protocol, flow["flows"], flow["packets"], flow["bytes"])
            flows.append(flow_one)
        mycursor.executemany(sql_update, flows)
        print(mycursor.rowcount, "records inserted successful。")
        mycursor.close()
        mydb.commit()    # 数据表内容有更新，必须使用到该语句
        mydb.close()

        end_time = time.time()
        consume_time = end_time - start_time
        consume_time = '{:0>2s}'.format(str(int(consume_time // 3600))) \
               + ':{:0>2s}'.format(str(int((consume_time // 60) % 60))) \
               + ':{:0>2s}'.format(str(int(consume_time % 60)))
        print(consume_time) 

1000133 记录插入成功。
00:01:14
1000414 记录插入成功。
00:01:20
1000623 记录插入成功。
00:01:08
1000830 记录插入成功。
00:01:22
1001038 记录插入成功。
00:01:48
1001239 记录插入成功。
00:01:27
1001565 记录插入成功。
00:01:39
1001741 记录插入成功。
00:01:43
1001998 记录插入成功。
00:01:47
1002259 记录插入成功。
00:02:02
1002533 记录插入成功。
00:01:51
1002801 记录插入成功。
00:02:03


In [6]:
import time
import json

# update 1 X 1000000 records at a time
# Use List to collect the time of each insert operation for every 1 million records
time_records = []
time_records_opt = []
for json_name in s:
    mydb.reconnect()
    mycursor = mydb.cursor()
    sql_update = "INSERT INTO p_netflow (srcaddr, srcport, first, last, protocol, flows, packets, bytes) VALUES (%s,%s,%s,%s,%s,%s,%s,%s) ON DUPLICATE KEY UPDATE `last` = VALUES(`last`), `flows`= VALUES(`flows`) + `flows`, `packets`= VALUES(`packets`) + `packets`, `bytes`= VALUES(`bytes`) + `bytes`"

    start_time = time.time()
    flows = []
    with open('data/netflow/' + json_name) as record:
        new_record = json.load(record)
        new_record.sort(key=lambda k: (k.get('srcaddr', 0)))
        for flow in new_record:
            srcaddr = struct.unpack('!I', socket.inet_aton(flow["srcaddr"]))[0]
            protocol = protocol_to_number(flow["protocol"])
            flow_one = (srcaddr, flow["srcport"], flow["First"], flow["Last"], protocol, flow["flows"], flow["packets"], flow["bytes"])
            flows.append(flow_one)
        mycursor.executemany(sql_update, flows)
        print(mycursor.rowcount, "records inserted successful。")
        mycursor.close()
        mydb.commit()    # 数据表内容有更新，必须使用到该语句
        mydb.close()
        end_time = time.time()
        consume_time = end_time - start_time
        time_records.append(consume_time)
        print(consume_time) 
        consume_time = '{:0>2s}'.format(str(int(consume_time // 3600))) \
               + ':{:0>2s}'.format(str(int((consume_time // 60) % 60))) \
               + ':{:0>2s}'.format(str(int(consume_time % 60)))
        time_records_opt.append(consume_time)
        print(consume_time) 

1000133 records inserted successful。
72.96194982528687
00:01:12
1000414 records inserted successful。
70.70519757270813
00:01:10
1000623 records inserted successful。
65.81352210044861
00:01:05
1000830 records inserted successful。
65.94893884658813
00:01:05
1001038 records inserted successful。
75.80351662635803
00:01:15
1001239 records inserted successful。
75.78497791290283
00:01:15
1001565 records inserted successful。
78.13914227485657
00:01:18
1001741 records inserted successful。
83.78260707855225
00:01:23
1001998 records inserted successful。
101.05721020698547
00:01:41
1002259 records inserted successful。
104.81750917434692
00:01:44
1002533 records inserted successful。
141.31164383888245
00:02:21
1002801 records inserted successful。
130.81323981285095
00:02:10
1002907 records inserted successful。
131.78951454162598
00:02:11
1003270 records inserted successful。
134.2951202392578
00:02:14
1003449 records inserted successful。
201.62654757499695
00:03:21
1003593 records inserted successfu

OperationalError: 2055: Lost connection to MySQL server at 'localhost:3306', system error: 32 Broken pipe

In [ ]:
import time
import json

# update 1 X 1000000 records at a time
# Use List to collect the time of each insert operation for every 1 million records
time_records = []
time_records_opt = []
for json_name in s[18:]:
    mydb.reconnect()
    mycursor = mydb.cursor()
    sql_update = "INSERT INTO p_netflow (srcaddr, srcport, first, last, protocol, flows, packets, bytes) VALUES (%s,%s,%s,%s,%s,%s,%s,%s) ON DUPLICATE KEY UPDATE `last` = VALUES(`last`), `flows`= VALUES(`flows`) + `flows`, `packets`= VALUES(`packets`) + `packets`, `bytes`= VALUES(`bytes`) + `bytes`"

    start_time = time.time()
    flows = []
    with open('data/netflow/' + json_name) as record:
        new_record = json.load(record)
        new_record.sort(key=lambda k: (k.get('srcaddr', 0)))
        for flow in new_record:
            srcaddr = struct.unpack('!I', socket.inet_aton(flow["srcaddr"]))[0]
            protocol = protocol_to_number(flow["protocol"])
            flow_one = (srcaddr, flow["srcport"], flow["First"], flow["Last"], protocol, flow["flows"], flow["packets"], flow["bytes"])
            flows.append(flow_one)
        mycursor.executemany(sql_update, flows)
        print(mycursor.rowcount, "records inserted successful。")
        mycursor.close()
        mydb.commit()    # 数据表内容有更新，必须使用到该语句
        mydb.close()
        end_time = time.time()
        consume_time = end_time - start_time
        time_records.append(consume_time)
        print(consume_time) 
        consume_time = '{:0>2s}'.format(str(int(consume_time // 3600))) \
               + ':{:0>2s}'.format(str(int((consume_time // 60) % 60))) \
               + ':{:0>2s}'.format(str(int(consume_time % 60)))
        time_records_opt.append(consume_time)
        print(consume_time) 